## Import libraries and check GPU
An Ampere architecture GPU (RTX 30xx) was used, which requires specific installation packages.

1. NVIDIA Cuda Toolkit 11.2
2. cuDNN 8.1.1
3. Visual studio windows
4. Python 3.8.6
5. tf-nightly build (2.5.0dev)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Conv3D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import time
import gc
import seaborn as sns
import math

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
print('keras: %s' % keras.__version__)

keras: 2.4.0


## CNN Creation

In [3]:
#activation=tf.keras.layers.LeakyReLU(alpha=0.05)
#activation='relu'

In [4]:
model = keras.models.Sequential()
layers = tf.keras.layers

model.add(layers.Conv3D(filters=32, kernel_size=(4,4,4), strides=2,padding="same", activation=tf.keras.layers.LeakyReLU(alpha=0.05), use_bias=False, input_shape=(32,32,32,1),data_format='channels_last'))
model.add(layers.Conv3D(filters=32, kernel_size=(4,4,4), strides=2,padding="same", activation=tf.keras.layers.LeakyReLU(alpha=0.05), use_bias=False))
model.add(layers.Conv3D(filters=32, kernel_size=(4,4,4), strides=2,padding="same", activation=tf.keras.layers.LeakyReLU(alpha=0.05), use_bias=False))
model.add(layers.Flatten())

model.add(layers.Dense(64,tf.keras.layers.LeakyReLU(alpha=0.05),use_bias = False))
model.add(layers.Dense(10,tf.keras.layers.LeakyReLU(alpha=0.05),use_bias = False))
model.add(layers.Dense(1,tf.keras.layers.LeakyReLU(alpha=0.05),use_bias = False))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 16, 16, 16, 32)    2048      
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 8, 8, 8, 32)       65536     
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 4, 4, 4, 32)       65536     
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                131072    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                640       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1

In [6]:
def custom_loss(y_true,y_predicted):
    error = 100*tf.math.abs((y_true-y_predicted)/y_true)*tf.math.tanh(30*y_true)
    return error

In [7]:
#Error as custom_loss or 'mape'

In [8]:
model.compile(optimizer=Adam(learning_rate=0.001,amsgrad=True),loss='mape')

## Import data
Large `.mat` files saved with `'-v7.3'` do not work with loadmat. 

Mat73 must be installed through `pip install mat73`.

Reshaping was done in `dataprocessing.ipynb` and the resulting 2.400.000x32x32x32x1 matrix is saved as `samples.npy`. This was split up into  segments (samples0-7) that fit in the RAM (32GB).


In [ ]:
ep=1
batch=20

for j in range(0,43):
    print('Set of Epoch '+str(1+ep*j)+'-'+str(ep*(j+1)))
    print('')
    
    for i in range(0,8):
        labels = np.load('data/labels'+str(i)+'.npy')
        samples = np.load('data/samples'+str(i)+'.npy')
        if i==7:
            model.fit(x=samples[0:200000],y=labels[0:200000],validation_split=0.001,batch_size=batch,epochs=ep,shuffle=True,verbose=1)
        else:
            model.fit(x=samples,y=labels,validation_split=0.001,batch_size=batch,epochs=ep,shuffle=True,verbose=1)
        gc.collect()
        time.sleep(10)
    
    samples = np.load('data/samples7.npy')
    labels = np.load('data/labels7.npy')
    score=model.evaluate(samples[200001:300000],labels[200001:300000], verbose=1)
    gc.collect()
    time.sleep(10)

Set of Epoch 1-1

3125/3125 [==============================] - 9s 3ms/step - loss: 19.0761
Set of Epoch 2-2

3125/3125 [==============================] - 8s 3ms/step - loss: 18.7388
Set of Epoch 3-3

3125/3125 [==============================] - 8s 3ms/step - loss: 16.0128
Set of Epoch 4-4

3125/3125 [==============================] - 8s 3ms/step - loss: 17.3223
Set of Epoch 5-5

3125/3125 [==============================] - 9s 3ms/step - loss: 15.3664
Set of Epoch 6-6

3125/3125 [==============================] - 9s 3ms/step - loss: 14.6976
Set of Epoch 7-7

3125/3125 [==============================] - 9s 3ms/step - loss: 14.6079
Set of Epoch 8-8

3125/3125 [==============================] - 9s 3ms/step - loss: 15.3377
Set of Epoch 9-9

3125/3125 [==============================] - 9s 3ms/step - loss: 15.5544
Set of Epoch 10-10

14985/14985 [==============================] - 68s 5ms/step - loss: 13.6797 - val_loss: 15.6326


3125/3125 [==============================] - 8s 3ms/step - loss: 14.7064
Set of Epoch 11-11

3125/3125 [==============================] - 9s 3ms/step - loss: 13.9632
Set of Epoch 12-12

3125/3125 [==============================] - 9s 3ms/step - loss: 14.0098
Set of Epoch 13-13

3125/3125 [==============================] - 8s 3ms/step - loss: 14.0831
Set of Epoch 14-14

3125/3125 [==============================] - 8s 3ms/step - loss: 13.8640
Set of Epoch 15-15

3125/3125 [==============================] - 8s 3ms/step - loss: 14.6847
Set of Epoch 16-16

3125/3125 [==============================] - 9s 3ms/step - loss: 13.7254
Set of Epoch 17-17

3125/3125 [==============================] - 9s 3ms/step - loss: 13.6860
Set of Epoch 18-18

3125/3125 [==============================] - 9s 3ms/step - loss: 14.0522
Set of Epoch 19-19

3125/3125 [==============================] - 9s 3ms/step - loss: 13.5542
Set of Epoch 20-20

14985/14985 [==============================] - 66s 4ms/step - loss: 12

3125/3125 [==============================] - 9s 3ms/step - loss: 14.3250
Set of Epoch 21-21

3125/3125 [==============================] - 8s 3ms/step - loss: 13.4780
Set of Epoch 22-22

3125/3125 [==============================] - 8s 3ms/step - loss: 15.2762
Set of Epoch 23-23

3125/3125 [==============================] - 8s 3ms/step - loss: 13.6346
Set of Epoch 24-24

3125/3125 [==============================] - 8s 3ms/step - loss: 13.3765
Set of Epoch 25-25

3125/3125 [==============================] - 8s 3ms/step - loss: 13.6144
Set of Epoch 26-26

3125/3125 [==============================] - 10s 3ms/step - loss: 13.4488
Set of Epoch 27-27

3125/3125 [==============================] - 10s 3ms/step - loss: 13.7654
Set of Epoch 28-28

9990/9990 [==============================] - 44s 4ms/step - loss: 12.1287 - val_loss: 11.1067


In [ ]:
predicted = model.predict(samples[200001:300000]).flatten()
truth = labels[200001:300000].flatten()

In [ ]:
ax = sns.scatterplot(x = predicted, y = truth)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim([np.min(predicted[predicted>0]),np.max(predicted)])
ax.set_ylim([np.min(truth),np.max(truth)])

In [ ]:
#np.savetxt('predicted.csv', predicted, delimiter=',')
#np.savetxt('truth.csv', truth, delimiter=',')